In [ ]:
#| default_exp vs

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import *

In [ ]:
#| hide
#| exporti
from fastcore.basics import patch

In [ ]:
#| export
import sys
import mechanicalsoup
from lxml import etree
from math import sqrt
from ouscope.core import Telescope
import datetime

In [ ]:
#| exporti
mech = mechanicalsoup.StatefulBrowser(soup_config={'features': 'lxml'})

In [ ]:
#| exporti
def prtMag(m):
    m=m.split()
    if m[0]=='-' :
        return None
    v=float(m[0])
    try :
        e=float(m[1][1:-1])
    except ValueError:
        e=0.0
    return '%f/%f' % (v,e)

dsgn=['u', 'b', 'v', 'rc', 'ic']

In [ ]:
#| export
def get_VS_sequence(vs, fov=60, maglimit=17, DEBUG = False):
    #fov*=sqrt(2)
    url="http://www.aavso.org/cgi-bin/vsp.pl?name=%s&ccdtable=on&fov=%d" % ("%20".join(vs.split()),fov)
    url="https://www.aavso.org/apps/vsp/photometry/?fov=%.1f&star=%s&Rc=on&B=on&maglimit=%.1f" % ( fov,
            '+'.join(vs.split()), maglimit)
    page = mech.open(url)
    html = ''.join([str(ln) for ln in page.soup])
    page.close()
    parser=etree.HTMLParser()
    #tree=etree.parse(open('rcp/sekw.html'),parser)
    tree=etree.fromstring(html,parser)

    try :
        var=' '.join(tree.xpath('//p//strong//text()')[0].split()[1:])
        ra=tree.xpath('//p//strong//text()')[1].split()[0]
        dec=tree.xpath('//p//strong//text()')[2].split()[0]
        seq=tree.xpath('//p//strong//text()')[3]
    except IndexError :
        #print('Cannot get AAVSO sequence for', vs)
        return None, None

    stars=[]

    if DEBUG :
        print('\nSequence %s for: %s ( ra: %s  dec: %s )' % (seq, var, ra, dec), file=sys.stderr)

    for tab in tree.xpath('//table//tbody')[0:1]:
        #print >> sys.stderr, 'Sequence:', seq
        for row in tab.xpath('./tr')[1:-2]:
            c=row.xpath('./td/text()')
            auid=c[0]
            lbl=row.xpath('./td/strong/text()')[0]
            ra=c[1].split()[0]
            ra_flt=float(c[1].split()[1][1:-2])
            dec=c[2].split()[0]
            dec_flt=float(c[2].split()[1][1:-2])
            #print(c, file=sys.stderr)
            #print(auid, lbl, ra, ra_flt, dec, dec_flt, file=sys.stderr)
    #        for d,m in zip(dsgn, (c[4], c[5], c[6], c[8], c[9])):
    #            s=prtMag(m)
    #            if s :
    #                print '%s=%s' % (d,s),
            #print c[-1]
            stars.append([auid, lbl, ra, ra_flt, dec, dec_flt] +
                        [v  for v in c[3:7]])
            #print c
            #print lbl, "ID:%s U:%s B:%s V:%s Rc:%s Ic:%s Cmnt:%s" %(c[0], c[4], c[5], c[6], c[8], c[9],c[-1])

    #print(' ', seq, fov, file=sys.stderr)
    return seq, stars

In [ ]:
VS = 'SS Cyg'
seq = get_VS_sequence(VS, 30, 16.5)

print(f'Sequence: {seq[0]} ({len(seq[1])} stars) for {VS}:')
for s in seq[1]:
    print(f'{s[0]} ({s[1]:>3})', end=' ')
    for v in s[6:]:
        print(f'{v:>14}', end=' ')
    print()

Sequence: X35373AOJ (8 stars) for SS Cyg:
000-BCP-217 ( 96) 10.653 (0.097)  9.630 (0.081)  1.023 (0.126)  9.091 (0.094) 
000-BCP-198 ( 98) 10.162 (0.080)  9.794 (0.071)  0.368 (0.107)  9.601 (0.071) 
000-BCP-194 (103) 10.712 (0.094) 10.279 (0.073)  0.433 (0.119)              — 
000-BCP-224 (108) 10.954 (0.110) 10.831 (0.080)  0.123 (0.136)              — 
000-BCP-202 (109) 11.451 (0.032) 10.898 (0.027)  0.553 (0.042) 10.591 (0.027) 
000-BCP-241 (114) 12.038 (0.032) 11.413 (0.026)  0.625 (0.041) 11.051 (0.033) 
000-BCP-207 (119) 12.486 (0.019) 11.866 (0.015)  0.620 (0.024) 11.507 (0.019) 
000-BDC-473 (123) 12.898 (0.021) 12.279 (0.020)  0.619 (0.029) 11.937 (0.024) 


In [ ]:
#| export
@patch
def submitVarStar(self: Telescope, name, expos=90, filt='BVR',comm='', tele='COAST'):
    o=SkyCoord.from_name(name)
    return self.submit_job_api(o, name=name, comment=comm,
                            exposure=expos*1000, filt=filt, tele=tele)

In [ ]:
#| login
if False :
    print("Submitting a VS job")
    rq = OSO.submitVarStar('V1223 Sgr', expos=180)
    if rq[0] :
        print("Waiting for job to be accepted")
        while (status:=OSO.get_request(int(rq[1]))['status'])!='Waiting' :
            print(status, end='\r')
            sys.stdout.flush()
            time.sleep(15)
        print(status)
        print("Cancelling the job")
        OSO.do_rm_api("0-cancel-request", {'rid':int(rq[1])})
        print("Waiting for job to be cancelled")
        while 'pending cancel' in (status:=OSO.get_request(int(rq[1]))['status']):
            print(status, end='\r')
            sys.stdout.flush()
            time.sleep(15)
        print(status)
    else :
        print('Submission failed')

In [ ]:
#| login
scope=Telescope(config='~/.config/telescope.ini')

In [ ]:
#| login
reqlst=scope.get_user_requests(sort='completion')
print(f"{'object':12} {'request time':19} {'id':12} {'status'}")
for rq in sorted(reqlst, key=lambda r: int(r['requesttime']), reverse=True)[:20]:
    print(f"{rq['objectname']:12}", 
          f"{datetime.datetime.fromtimestamp(int(rq['requesttime']))}",
          f"{rq['id']:12}",
          f"{Telescope.REQUESTSTATUS_TEXTS[int(rq['status'])]}"
         )

object       request time        id           status
LX Cyg       2023-12-08 22:51:03 759783       Waiting
CH Cyg       2023-12-08 22:51:02 759780       Waiting
DX Vul       2023-12-08 22:51:02 759782       Waiting
SS Cyg       2023-12-08 22:51:02 759781       Waiting
EQ Lyr       2023-12-07 22:36:16 759658       Complete
DQ Vul       2023-12-07 22:16:53 759655       Complete
IP Cyg       2023-12-07 22:16:52 759653       Complete
V686 Cyg     2023-12-07 22:16:52 759654       Complete
LX Cyg       2023-12-04 10:41:24 759483       Complete
EQ Lyr       2023-12-03 22:45:58 759452       Complete
DQ Vul       2023-12-03 22:45:58 759451       Complete
DX Vul       2023-12-03 22:45:57 759450       Complete
IP Cyg       2023-12-03 22:45:57 759448       Complete
V686 Cyg     2023-12-03 22:45:57 759449       Complete
SS Cyg       2023-12-03 22:45:57 759447       Complete
CH Cyg       2023-12-03 22:45:56 759446       Complete
LX Cyg       2023-11-30 09:58:45 759188       Complete
SS Cyg       202